In [42]:
import pandas as pd
file="/home/davide/Downloads/geniregolatiLexAincianobatteri.xlsx"
df_evi=pd.read_excel(file)
df_evi["old_locus_tag"]=df_evi["old_locus_tag"].str.split("-")
df_evi=df_evi.explode("old_locus_tag")
df_evi["old_locus_tag"]=df_evi["old_locus_tag"].str.split()
df_evi=df_evi.explode("old_locus_tag")
df_evi

,old_locus_tag,gene_omologo,motivo_omologo,Organismo/evidenze,Pubblicazione
0,all0875,glgB,AGTataaaACT,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018"
1,all2563,talB,AGTcttaacaaACT,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018"
2,all4713,galE,AGTagttgtcaACT,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018"
3,all4121,petH,AGTTagaaggtgAACa,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018"
4,alr4641,prxA,ACAGaTcgtaatcaAACTGT,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018"
...,...,...,...,...,...
181,all4148,"petF, fd1",AGTatcagaattgaACT\nAGTtatatACT\nAGTataaaaaaaACT,Anabaena PCC 7120: mista,"Srivastava et al., 2022"
182,all1430,fdxH,AGTgtccaACT,Anabaena PCC 7120: mista,"Srivastava et al., 2022"
183,all2919,fd2,AGTttgggtgaaaACT\nAGTttcacccaACT,Anabaena PCC 7120: mista,"Srivastava et al., 2022"
184,alr0784,fd3,AGTagttgACT\nAGTtacttcACT,Anabaena PCC 7120: mista,"Srivastava et al., 2022"


In [43]:
import gspread
folder_id = "1YDMwApF3vLdwxUzBXBZZJWsEqzM0GOoU"
gc = gspread.oauth()
worksheet_dict = gc.list_spreadsheet_files(folder_id=folder_id)
worksheet_list = [el['name'] for el in worksheet_dict]


In [ ]:
from pathlib import Path
dir=Path("/home/davide/Desktop/genomiChro/blastvsPCC7120")
#iterate over all the files in the directory
for file in dir.iterdir():
    #read the file
    df=pd.read_csv(file,sep="\t",header=None)
    #set the columns name
    df.columns=["protein_id","subject","identity","length","mismatch","gapopen","qstart","qend","sstart","send","evalue","bitscore"]
    #drop the columns that are not useful
    df.drop(["mismatch","gapopen","qstart","qend","sstart","send","bitscore"],axis=1,inplace=True)
    #create a new column with the old_locus_tag
    df.loc[:,"old_locus_tag"]=df.loc[:,"subject"].str.extract(r"\((.*?)\)",expand=False)
    #keep only the first hit for each query
    #df.drop_duplicates(subset="query",keep="first",inplace=True)
    #order the dataframe by evalue
    df.sort_values(by="evalue",inplace=True)
    #drop duplicates of old_locus_tag keeping the first with the lowest evalue
    df.drop_duplicates(subset="old_locus_tag",keep="first",inplace=True)
    #create a new column with the information of the blastp
    df["info_blastp"]='id:'+df["identity"].astype(str)+"%, len:"+df["length"].astype(str)+", evalue:"+df["evalue"].astype(str)
    #drop the columns that are not useful
    df.drop(columns=["subject","identity","length","evalue"],inplace=True)
    #save the dataframe in a new file
    #join with df_evi
    df=pd.merge(df,df_evi,how="inner",on="old_locus_tag")
    for name in worksheet_list:
        if name in file.name:
            worksheet=gc.open(name).sheet1
            df_sheet=pd.DataFrame(worksheet.get_all_records())
            df_join=pd.merge(df_sheet,df,how="left",on="protein_id")
            df_join=df_join[
                ['', 'gene', 'product', 'motivo', 'motivo_omologo', 'gene_omologo','old_locus_tag',
       'Organismo/evidenze', 'Pubblicazione','info_blastp' , 'location', 'pathway', 'ko_brite',
       'protein_id', 'GO_component', 'GO_process', 'GO_function',
                 ]]
            df_join.to_excel(f"/home/davide/Downloads/{name}.xlsx",index=False)
            #print len of the two dataframes
            print(len(df),len(df_sheet),len(df_join))

    
    
    



In [31]:
df_sheet[df_sheet.duplicated(subset="protein_id",keep=False)]

,Unnamed: 0,gene,product,motivo,location,pathway,ko_brite,protein_id,GO_component,GO_process,GO_function,motivo_omologo,gene_omologo,info_blastp,Evidenze,Pubblicazione
0,Trasportatori,,,,,,,,,,,,,,,
8,,,YnfA family protein,TTGTGCAGTTGCACTA,[82392:82722](-),,['ko02000 - Transporters'],WP_015162980.1,GO:0016020 - membrane [Evidence IEA],,,,,,,
12,,,YnfA family protein,GAGTACGAGTGAAATA,[82392:82722](-),,['ko02000 - Transporters'],WP_015162980.1,GO:0016020 - membrane [Evidence IEA],,,,,,,
14,DNA riparo e ricombinazione,,,,,,,,,,,,,,,
26,Sistema a due componenti,,,,,,,,,,,,,,,
33,Fattori di trascrizione,,,,,,,,,,,,,,,
38,Biogenesi del tRNA,,,,,,,,,,,,,,,
43,Fotosintesi,,,,,,,,,,,,,,,
47,Sintesi peptidoglicano,,,,,,,,,,,,,,,
51,Altro,,,,,,,,,,,,,,,


In [32]:
#print duplicates of protein_id
df_join[df_join.duplicated(subset="protein_id",keep=False)]
#drop dupkicates of protein_id that have different locus_tag


,Unnamed: 0,gene,product,motivo,location,pathway,ko_brite,protein_id,GO_component,GO_process,...,gene_omologo_x,info_blastp_x,Evidenze,Pubblicazione_x,old_locus_tag,info_blastp_y,gene_omologo_y,motivo_omologo_y,Organismo/evidenze,Pubblicazione_y
0,Trasportatori,,,,,,,,,,...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
8,,,YnfA family protein,TTGTGCAGTTGCACTA,[82392:82722](-),,['ko02000 - Transporters'],WP_015162980.1,GO:0016020 - membrane [Evidence IEA],,...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
12,,,YnfA family protein,GAGTACGAGTGAAATA,[82392:82722](-),,['ko02000 - Transporters'],WP_015162980.1,GO:0016020 - membrane [Evidence IEA],,...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
14,DNA riparo e ricombinazione,,,,,,,,,,...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
26,Sistema a due componenti,,,,,,,,,,...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
33,Fattori di trascrizione,,,,,,,,,,...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
38,Biogenesi del tRNA,,,,,,,,,,...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
43,Fotosintesi,,,,,,,,,,...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
47,Sintesi peptidoglicano,,,,,,,,,,...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
51,Altro,,,,,,,,,,...,,,,,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df_join.columns
#reorder the columns

df_join=df_join[
['', 'gene', 'product', 'motivo', 'motivo_omologo', 'gene_omologo','old_locus_tag',
       'Organismo/evidenze', 'Pubblicazione','info_blastp' , 'location', 'pathway', 'ko_brite',
       'protein_id', 'GO_component', 'GO_process', 'GO_function',
    ]]
df_join

,,gene,product,motivo,motivo_omologo,gene_omologo,old_locus_tag,Organismo/evidenze,Pubblicazione,info_blastp,location,pathway,ko_brite,protein_id,GO_component,GO_process,GO_function
0,DNA riparo e ricombinazione,,,,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,
1,,,FAD-binding domain-containing protein,CAGTATAAATGTACTA,NaN,NaN,NaN,NaN,NaN,NaN,[11864:13433](-),,['ko03400 - DNA repair and recombination prote...,WP_106547875.1,,,
2,,,single-stranded DNA-binding protein,AAGTACAATTGTACTA,AGTActtatgTACT,ssb1,alr0088,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018","id:72.973%, len:111, evalue:1.46e-57",[24608:24977](-),"['DNA replication', 'Mismatch repair', 'Homolo...","['ko03032 - DNA replication proteins', 'ko0340...",WP_106545218.1,,,
3,,lexA,transcriptional repressor LexA,TAGTACGACTGTTCTA,TAGTactaatgttCTA,lexA,alr4908,Anabaena PCC 7120: sperimentale,"Kumar et al., 2018","id:77.228%, len:202, evalue:6.15e-117",[6157:6769](-),,"['ko01002 - Peptidases and inhibitors', 'ko034...",WP_015155906.1,,GO:0006282 - regulation of DNA repair [Evidenc...,GO:0004252 - serine-type endopeptidase activit...
4,,sbcC,exonuclease subunit SbcC,CAGTGCATCTGTACTG,AGTgcatctgtACT,sbcC,alr3988,Anabaena PCC 7120: Bioinformatica,"Kumar et al., 2018","id:65.805%, len:1006, evalue:0.0",[25310:28337](+),,['ko03400 - DNA repair and recombination prote...,WP_106545523.1,,GO:0006260 - DNA replication [Evidence IEA],GO:0004527 - exonuclease activity [Evidence IEA]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,,,hypothetical protein,TAGTAAATATACTCTT,NaN,NaN,NaN,NaN,NaN,NaN,[1419:1599](+),,[],WP_015157212.1,,,
205,,,hypothetical protein,TAGTAAACTTGATATA,NaN,NaN,NaN,NaN,NaN,NaN,[6591:6903](-),,[],WP_015156495.1,,,
206,,,hypothetical protein,CAGAAAAGCTTAACTA,NaN,NaN,NaN,NaN,NaN,NaN,[29891:30314](+),,[],WP_106546339.1,,,
207,,,hypothetical protein,TATAAAATTTTCTCTA,NaN,NaN,NaN,NaN,NaN,NaN,[22035:22257](-),,[],WP_106546970.1,,,
